In [2]:
import skorch
import numpy as np
import torch
from torch import nn
import os

In [3]:
class LSTMNet(nn.Module):
    def __init__(self, in_features, hidden_size, out_features):
        super(LSTMNet, self).__init__()
        self.in_features = in_features
        self.hidden_size = hidden_size
        self.out_features = out_features
        self.linear = nn.Linear(in_features=in_features, out_features=hidden_size * 2, dtype=torch.double)
        self.lstm = nn.LSTM(in_features, hidden_size, num_layers=2, bidirectional=True, dropout=0.5, dtype=torch.double)
        # self.softmax = nn.Sequential(
        #                 nn.Linear(in_features=hidden_size * 2, out_features=out_features),
        #                 nn.Softmax(dim=1)
        #             )
        self.linear2 = nn.Sequential(
                        nn.Dropout(0.5),
                        nn.Linear(in_features=hidden_size * 2, out_features=out_features, dtype=torch.double)
                       )

    def forward(self, input):
        # linear_output = self.linear(input)
        # lstm_output = self.lstm(input.reshape(-1, LENGTH_OF_SIGNAL, self.in_features))
        linear_output = self.linear(input)
        lstm_output,_ = self.lstm(input.reshape(-1,1,self.in_features))
        lstm_output = lstm_output.reshape(-1, self.hidden_size * 2)
        # output = self.softmax(linear_output + lstm_output)
        output = self.linear2(linear_output + lstm_output)
        return output

    
def predict(model, x, out_features):
    tag = model(x)
    y=torch.zeros((tag.shape[0],), dtype=int)
    for i in range(tag.shape[0]):
        mx=tag[i][0]
        for j in range(1,tag.shape[1]):
            if(tag[i][j]>mx):
                mx=tag[i][j]
                y[i]=j
    return y


In [ ]:
import scoring
import importlib
from tqdm.notebook import tqdm
importlib.reload(scoring)
feature_number = 105
X = []
y = []

model = LSTMNet(in_features=feature_number, hidden_size=20, out_features=7)


for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X.append(file['features'])
        y.append(file['labels'])

num_file = len(X)
# print(num_file)

X_train = X[num_file//4:num_file]
y_train = y[num_file//4:num_file]

X_test = X[0:num_file//4]
y_test = y[0:num_file//4]

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
loss_function = nn.CrossEntropyLoss()
max_epochs = 50
batch_size = 32

for epoch in range(max_epochs):
    print("epoch: ", epoch)
    sum_loss = 0
    array = np.arange(len(X_train))
    slice=np.random.choice(array.shape[0],batch_size)
    for i in tqdm(slice, colour="pink"):
    # for i in range(len(X_train)):
        sentence = torch.DoubleTensor(X_train[i])
        target = torch.LongTensor(y_train[i])
        # f=[0,0,0,0,0,0,0,0]
        # for t in target:
        #     f[t]=1
        # print(f)
        tag_score = model(sentence)
        loss = loss_function(tag_score, target)
        loss.backward()
        optimizer.step()

    y_pred = np.zeros((0,))
    y_true = np.zeros((0,))
    for i in tqdm(range(len(X_train))):
        y_pred=np.concatenate((y_pred, predict(model, torch.DoubleTensor(X_train[i]), out_features=7)))
        y_true = np.concatenate((y_true, y_train[i]))
    scoring.score(y_pred, y_true)

epoch:  0


  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

Class 0: Precision: 0.718, Recall: 1.000
Class 1: Precision: 0.000, Recall: 0.000
Class 2: Precision: 0.000, Recall: 0.000
Class 3: Precision: 0.000, Recall: 0.000
Class 4: Precision: 0.000, Recall: 0.000
Class 5: Precision: 0.000, Recall: 0.000
Class 6: Precision: 0.000, Recall: 0.000
mf1 score: nan
acc_score: 0.718
kappa_score: 1.117
epoch:  1


E:\2023 Fall\IML\project\iml-project\scoring.py:27: RuntimeWarning: invalid value encountered in scalar divide
  pre += TP / (TP + FP)


...

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

Class 0: Precision: 0.718, Recall: 1.000
Class 1: Precision: 0.000, Recall: 0.000
Class 2: Precision: 0.000, Recall: 0.000
Class 3: Precision: 0.000, Recall: 0.000
Class 4: Precision: 0.000, Recall: 0.000
Class 5: Precision: 0.000, Recall: 0.000
Class 6: Precision: 0.000, Recall: 0.000
mf1 score: nan
acc_score: 0.718
kappa_score: 1.117
epoch:  3


  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]